# 🚀 ChatRoutes Python SDK - Quick Start Guide

[![PyPI](https://img.shields.io/pypi/v/chatroutes.svg)](https://pypi.org/project/chatroutes/)
[![Python](https://img.shields.io/pypi/pyversions/chatroutes.svg)](https://pypi.org/project/chatroutes/)

Welcome to ChatRoutes! This notebook will help you get started with the ChatRoutes Python SDK.

## ⚠️ Beta Release Notice

ChatRoutes is currently in **beta**. The API may change without maintaining backward compatibility. Please use with caution in production environments.

## 🔑 Getting Your API Key

**IMPORTANT:** Before you begin, you need to get your API key:

1. **Visit** [chatroutes.com](https://chatroutes.com)
2. **Sign up** for a free account
3. **Go to Dashboard** → Navigate to the **API section**
4. **Generate** your API key
5. **Copy** the API key and paste it below when prompted

## 🤖 Supported Models

Currently, ChatRoutes supports the following AI models:
- **`gpt-5`** - OpenAI's GPT-5
- **`claude-opus-4.1`** - Anthropic's Claude Opus 4.1

---

## 📦 Step 1: Install ChatRoutes SDK

In [ ]:
!pip install -q chatroutes
print("✅ ChatRoutes SDK installed successfully!")

## 🔐 Step 2: Set Up Your API Key

Enter your API key below. You can get it from [chatroutes.com](https://chatroutes.com) → Dashboard → API section.

In [ ]:
from getpass import getpass
from chatroutes import ChatRoutes

# Enter your API key (it will be hidden)
api_key = getpass('Enter your ChatRoutes API key: ')

# Initialize the ChatRoutes client
client = ChatRoutes(api_key=api_key)

print("✅ ChatRoutes client initialized successfully!")

## 💬 Step 3: Create Your First Conversation

Let's create a conversation and send a message!

In [ ]:
# Create a new conversation
conversation = client.conversations.create({
    'title': 'My First ChatRoutes Conversation',
    'model': 'gpt-5'  # or 'claude-opus-4.1'
})

print(f"✅ Created conversation: {conversation['title']}")
print(f"📝 Conversation ID: {conversation['id']}")
print(f"🤖 Model: {conversation['model']}")

## 📤 Step 4: Send a Message

In [ ]:
# Send a message to the conversation
response = client.messages.send(
    conversation['id'],
    {
        'content': 'Hello! Can you explain what ChatRoutes is in one sentence?',
        'model': 'gpt-5'
    }
)

print("\n💬 Your message:")
print(response['userMessage']['content'])
print("\n🤖 AI Response:")
print(response['assistantMessage']['content'])

## 🌊 Step 5: Try Streaming Responses

ChatRoutes supports streaming for real-time responses!

In [ ]:
import sys

print("🌊 Streaming response:\n")

# Define callback for streaming chunks
def on_chunk(chunk):
    if chunk['choices'] and chunk['choices'][0]['delta'].get('content'):
        content = chunk['choices'][0]['delta']['content']
        print(content, end='', flush=True)

def on_complete(response):
    print("\n\n✅ Streaming complete!")

# Send a message with streaming
client.messages.stream(
    conversation['id'],
    {
        'content': 'Tell me a short joke about programming.',
        'model': 'gpt-5'
    },
    on_chunk=on_chunk,
    on_complete=on_complete
)

## 🌳 Step 6: Create a Branch (Alternative Response)

One of ChatRoutes' unique features is conversation branching - explore alternative responses!

In [ ]:
# Create a branch for alternative responses
branch = client.branches.create(
    conversation['id'],
    {
        'title': 'Alternative Response',
        'contextMode': 'FULL'
    }
)

print(f"✅ Created branch: {branch['title']}")
print(f"🌿 Branch ID: {branch['id']}")

# Send a message in the branch
branch_response = client.messages.send(
    conversation['id'],
    {
        'content': 'Now tell me a joke about AI instead.',
        'model': 'gpt-5',
        'branchId': branch['id']
    }
)

print("\n🌿 Branch response:")
print(branch_response['assistantMessage']['content'])

## 🎭 Step 7: Try Claude Model

Let's create a conversation using Claude!

In [ ]:
# Create a conversation with Claude
claude_conversation = client.conversations.create({
    'title': 'Testing Claude Opus',
    'model': 'claude-opus-4.1'
})

# Send a message to Claude
claude_response = client.messages.send(
    claude_conversation['id'],
    {
        'content': 'Write a haiku about artificial intelligence.',
        'model': 'claude-opus-4.1'
    }
)

print("🎭 Claude's response:")
print(claude_response['assistantMessage']['content'])

## 📋 Step 8: List Your Conversations

In [ ]:
# List all conversations
result = client.conversations.list({
    'page': 1,
    'limit': 5,
    'filter': 'all'
})

print(f"📊 Total conversations: {result['total']}")
print(f"📄 Page {result['page']} of {result['pages']}\n")

for conv in result['data']:
    print(f"💬 {conv['title']}")
    print(f"   ID: {conv['id']}")
    print(f"   Model: {conv['model']}")
    print(f"   Created: {conv['createdAt']}")
    print()

## 🌲 Step 9: View Conversation Tree

See the structure of your conversation with all its branches!

In [ ]:
# Get conversation tree
tree = client.conversations.get_tree(conversation['id'])

def print_tree(node, indent=0):
    prefix = "  " * indent
    message_type = "🧑" if node['message']['role'] == 'user' else "🤖"
    content = node['message']['content'][:50] + "..." if len(node['message']['content']) > 50 else node['message']['content']
    print(f"{prefix}{message_type} {content}")
    
    for child in node.get('children', []):
        print_tree(child, indent + 1)

print("🌲 Conversation Tree:\n")
if tree['root']:
    print_tree(tree['root'])
else:
    print("No messages in conversation yet.")

## 🧹 Optional: Clean Up

Delete the test conversations if you want to clean up.

In [ ]:
# Uncomment to delete the conversations
# client.conversations.delete(conversation['id'])
# client.conversations.delete(claude_conversation['id'])
# print("✅ Conversations deleted")

print("ℹ️ Uncomment the code above to delete the test conversations")

## 📚 Next Steps

Congratulations! You've learned the basics of ChatRoutes. Here are some resources to help you go further:

### 📖 Documentation
- [GitHub Repository](https://github.com/chatroutes/chatroutes-python-sdk)
- [API Documentation](https://docs.chatroutes.com)
- [Quick Start Guide](https://github.com/chatroutes/chatroutes-python-sdk/blob/main/QUICKSTART.md)

### 💡 Examples
- [Basic Usage](https://github.com/chatroutes/chatroutes-python-sdk/blob/main/examples/basic_usage.py)
- [Streaming](https://github.com/chatroutes/chatroutes-python-sdk/blob/main/examples/streaming_example.py)
- [Branching](https://github.com/chatroutes/chatroutes-python-sdk/blob/main/examples/branching_example.py)
- [Error Handling](https://github.com/chatroutes/chatroutes-python-sdk/blob/main/examples/error_handling.py)

### 🤖 Supported Models

- **`gpt-5`** - OpenAI's GPT-5
- **`claude-opus-4.1`** - Anthropic's Claude Opus 4.1

### 🔧 Key Features to Explore

1. **Multiple Models**: Switch between GPT-5 and Claude Opus 4.1
2. **Conversation Branching**: Explore alternative conversation paths
3. **Streaming**: Real-time response streaming
4. **Message History**: Access and manage conversation history
5. **Branch Merging**: Merge branches back into main conversation

### 💬 Support
- Email: support@chatroutes.com
- GitHub Issues: [Report a bug](https://github.com/chatroutes/chatroutes-python-sdk/issues)

---

**Happy coding with ChatRoutes!** 🚀